In [1]:
from colbert.infra import ColBERTConfig
from colbert.modeling.checkpoint import Checkpoint

ckpt = Checkpoint("jinaai/jina-colbert-v2", colbert_config=ColBERTConfig())

/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Nov 22, 12:42:33] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


In [2]:
texts = [
    "- workspaceは、作業用のコンテナでPythonやvespa-cliが入っています。後ほど中身を説明します",
    "与えられた集合の中で与えられた点に最も近い点を見つけるという最適化問題である最近傍探索(Nearest Neighbor Search)というものがあります。また、近い順にk個の解をみつける場合は、kNN Searchと言います。ANN(Approximate Nearest Neighbor: 近似最近傍探索)とは、kNN Searchの一種で、厳密な最近傍を求めず、ある程度の精度で解を見つけることを許容することで、高速に解を求める手法です。",
    "ここでANNの話に戻りますが、ドキュメントのベクトルごとにクエリとの類似度を計算していくと、全てのドキュメントに対して計算をする必要があるため時間がかかります。検索エンジンの多くのユースケースでは検索にかかかる時間は短い方が好ましいです。そのため、高速に検索結果を取得できるようにANNが用いられることになります。"
]

ids = list(range(len(texts)))
embeddings = ckpt.docFromText(texts, bsize=8)

/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [3]:
from tqdm import tqdm
import torch

# ColBERTモデルからembeddingを取得（進捗バー付き）
bsize = 8
embeddings = []  # 各バッチの結果を順次蓄積

# tqdmのprogress bar
for i in tqdm(range(0, len(texts), bsize), desc="🔍 Encoding documents with ColBERT"):
    batch = texts[i:i + bsize]

    # ColBERTでバッチ埋め込み生成
    embs = ckpt.docFromText(batch, bsize=bsize, keep_dims=True)

    # docFromTextがtupleを返す場合は中身を取り出す
    if isinstance(embs, tuple):
        embs = embs[0]

    # GPUならCPUへ移動しておく
    if isinstance(embs, torch.Tensor) and embs.is_cuda:
        embs = embs.cpu()

    embeddings.append(embs)

# 最後に全バッチを連結して [N, L, H] に
embeddings = torch.cat(embeddings, dim=0)

print("✅ Done:", embeddings.shape)


🔍 Encoding documents with ColBERT:   0%|          | 0/1 [00:00<?, ?it/s]/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/satoshi.takatori.001/Desktop/playground/weaviate-sample/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
🔍 Encoding documents with ColBERT: 100%|██████████| 1/1 [00:00<00:00,  4.36it/s]

✅ Done: torch.Size([3, 117, 128])


In [7]:
print(embeddings[0])
print(len(embeddings[0]))
print(len(embeddings))

tensor([[ 0.1123,  0.0942, -0.0858,  ..., -0.1166, -0.0990, -0.0819],
        [ 0.1411,  0.0792, -0.1317,  ..., -0.0752, -0.0169, -0.0701],
        [ 0.0000,  0.0000, -0.0000,  ...,  0.0000, -0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [ 0.0000,  0.0000, -0.0000,  ..., -0.0000, -0.0000, -0.0000]])
117
3
